In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
import json
import math
from nltk.util import pr
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import os
Size = 0


In [2]:

# functions

def make_idf(lexicon):
    idf = dict()
    global Size
    N = Size
    for term,docs in lexicon.items():
        df = len(docs)
        temp = math.log(N / df)
        idf[term] = temp
    return idf

def cleaner(uncleaned):
    porter = PorterStemmer()
    lemma = WordNetLemmatizer()
    uncleaned = uncleaned.lower()
    uncleaned = uncleaned.strip()
    uncleaned = uncleaned.translate(
        {ord(i): None for i in '!\\@#-_:$%^&*();.,?/1”2’3“4‘567890\'\"'})
    for i in uncleaned:
        t = ord(i)
        if t < 97 or t > 122:
            uncleaned = uncleaned.replace(i, "")
    uncleaned = porter.stem(uncleaned)
    uncleaned = lemma.lemmatize(uncleaned, pos='v')
    return uncleaned

def Read_Data_From_File():
    Tokens = []
    alpha = ''
    Raw_Data = dict()
    X_headline = []
    Y_Sarcastic = []
    i = 0
    count = 0
    lexicon = {}

    stop_word = []
    with open("Stopword-List.txt", 'r') as stop:
        for line in stop:
            temp = line.strip()
            stop_word.append(temp)

    for lines in open('Sarcasm_Headlines_Dataset.json', 'r'):
        count += 1
        Raw_Data[i] = json.loads(lines)
        X_headline.append(Raw_Data[i]["headline"])
        Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])

        #tokenize here
        words = word_tokenize(X_headline[i])
        curr = str(i)
        for word in words:
            if len(word) >= 3:
                if word in stop_word:
                    continue
                taggs = nltk.tag.pos_tag(word)
                if(taggs[0][1] != 'NNP' and taggs[0][1] != 'FW' and taggs[0][1] != 'PRP'):
                    word = cleaner(word)
                    if len(word) >= 3:                 
                        # Tokens.append(word)

                        if word in lexicon:
                            if curr in lexicon[word]:
                                lexicon[word][i] += 1
                            else:
                                lexicon[word][i] = 1
                        else:
                            lexicon[word] = {}
                            if curr in lexicon[word]:
                                lexicon[word][i] += 1
                            else:
                                lexicon[word][i] = 1
        i = i+1
        # print(lexicon)
    global Size 
    Size = count
    return lexicon

def tf_idf_lexicon(lexicon, idf):
    tf_idf = {}
    for term, docs in lexicon.items():
        tf_idf[term] = {}
        for docNo, tf in docs.items():
            tf_idf[term][docNo] = tf * idf[term]

    return tf_idf

def applyPCA(df):
    print("In function")
    scaler = StandardScaler()
    #scaler=MinMaxScaler()
    print("Standard scaler defined")
    scaler.fit(df)
    print("scaler Fit defined")
    scaled_data = scaler.transform(df)
    print("scaler trasnform")

    pca = PCA(n_components=900)
    pca.fit(scaled_data)
    x_pca = pca.transform(scaled_data)
    print("Data Variance: ", sum(pca.explained_variance_ratio_)*100)

    return x_pca, scaled_data


def sort(tfidf):
    size = 26709
    list1 = []
    check = False
    for i in range(size):
        list1.append(i+1)
    # print(list1)
    
    for word,doc in tfidf.items():
        print(word)
        for i in list1:
            if i not in doc.keys():
                tfidf[word][i] = 0
    return tfidf

def make_idf1(Tokens):
    list1 = []
    idf = dict()
    global Size
    N = Size
    # for term,docs in lexicon.items():
    #     df = len(docs)
    #     temp = math.log(N / df)
    #     idf[term] = temp
    # return idf
    for i in Tokens:
        # print(i[0],i[1])        
        count =  i[0]
        while i[0]+1 == count:
             list1.append(Tokens[i[1]])
    print(list1)


In [3]:
# lexicon = Read_Data_From_File()
# print(lexicon)
# idf = make_idf(lexicon)
# print("Idf made")
# tf_idf = tf_idf_lexicon(lexicon,idf)
# print("tf_Idf made")
# # tf_idf = sort(tf_idf)
# with open("TFIDF_Dictionary.json", "w") as f: # Writing the index to JSON File.
#    f.write(json.dumps(tf_idf, sort_keys=False, indent=4)) 
with open('TFIDF_Dictionary.json') as json_file:
    Dict = json.load(json_file)
# Dict = sort(Dict)
# print(Dict)
# with open("TFIDF_Dictionary_new.json", "w") as f:
    # f.write(json.dumps(Dict, sort_keys=False, indent=4))

#Create Panda DataFrame
#    print("Hello")
df = pd.DataFrame(Dict.items())
vocab = list(Dict.keys())
vocab_len = len(vocab)
word_to_inx = {}
for word in vocab:
    word_to_inx[word] = vocab.index(word)
i = 0
dataset = np.empty((26709, vocab_len))
for line in open('Sarcasm_Headlines_Dataset.json', 'r'):
    data = json.loads(line)
    str_i = str(i)
    docVector = np.zeros(vocab_len)
    for word in data["headline"].split(" "):
        if word in word_to_inx and str_i in Dict[word]:
            wordPos = word_to_inx[word]
            docVector[wordPos] = Dict[word][str_i]
            dataset[i] = docVector
    i += 1

print(dataset.shape)

# df = np.transpose(df)
# print(df[0])

print(vocab_len)

print("Data Processing Completed........")
print("")
print("Applying PCA.....................")

x_pca, scaled_data = applyPCA(dataset)

print(x_pca,scaled_data)
print("")
print("PCA Applied......................")
print("")
print("Traning Model....................")

(26709, 18622)
18622
Data Processing Completed........

Applying PCA.....................
In function
Standard scaler defined
scaler Fit defined
scaler trasnform
Data Variance:  22.363574441852904
[[-5.38722640e-02 -3.07770974e-02  4.58026224e-03 ... -5.18182851e-01
   1.66149708e+00 -2.00327479e-01]
 [-3.97918282e-02 -1.44887513e-02 -3.78601933e-02 ... -3.32642575e-01
  -1.82867024e-01 -1.56176632e-02]
 [-8.34543482e-02 -4.57898260e-02 -2.62328096e-02 ...  5.92836469e+00
   1.38415095e+00  2.00209451e-01]
 ...
 [-3.34488689e-02 -1.00313559e-02 -3.62179674e-02 ... -2.10659051e-01
   1.33084296e-01 -6.56137467e-02]
 [-6.23458095e-02 -1.98454765e-02  1.07939651e-02 ... -2.23850604e+00
  -5.17559331e-02 -1.62762250e+00]
 [-3.02392648e-02 -6.45818761e-03 -3.72011666e-02 ...  5.25561592e-02
   1.86405192e-01  1.95317082e-02]] [[ 1.59176452e+01  0.00000000e+00  2.49026010e+01 ...  0.00000000e+00
  -6.11898398e-03  0.00000000e+00]
 [-6.28233629e-02  0.00000000e+00 -4.01564480e-02 ...  0.00000

In [4]:
X=[]
Y=[]

Raw_Data = dict()
X_headline = []
Y_Sarcastic = []
i = 0
count = 0

for lines in open('Sarcasm_Headlines_Dataset.json', 'r'):
    count += 1
    Raw_Data[i] = json.loads(lines)
    X_headline.append(Raw_Data[i]["headline"])
    Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])

size=len(X_headline)


In [5]:
len(Y_Sarcastic)

26709

In [6]:
#x_pca.to_csv('pData.csv')
#x_pca=pd.read_csv('pData')

#BreakData
X_train, X_test, y_train, y_test = train_test_split(x_pca, Y_Sarcastic[:size],test_size=0.20,random_state=48)

print("Applying SVM.....................")





Applying SVM.....................


In [7]:
#Apply SVM
model = svm.SVC(random_state=20)
print("Model Trained....................")
print("")
print("Applying Model...................")



Model Trained....................

Applying Model...................


In [8]:
model.fit(X_train,y_train)


SVC(random_state=20)

In [9]:
acc=model.score(X_test,y_test)

print("")
print("Accuracy= ",acc*100,"%")


Accuracy=  70.47922126544366 %


In [16]:
import sklearn.externals.joblib as extjoblib
import joblib
joblib.dump(clf, 'filename.pkl')

ModuleNotFoundError: No module named 'sklearn.externals.joblib'

In [10]:
##Test
# pca=PCA(n_components=2)
# pca.fit(scaled_data)
# x_pca=pca.transform(scaled_data)

# plt.figure(figsize=(20,12))
# plt.scatter(x_pca[:,0],x_pca[:,1])
# plt.xlabel('First principle component')
# plt.ylabel('Second principle component')